In [1]:
from glob import glob
import numpy as np
import os
import pickle
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
NEIGHBOR_THRESHOLD = 30
DEFAULT_DISTANCE = 100

In [3]:
with open("./train/0.pkl", "rb") as f:
    data = pickle.load(f)

In [3]:
def normalize_path(path):
    shift = path[0]
    path_centered = path - shift
    theta = -np.arctan2(path_centered[-1, 1], path_centered[-1, 0])
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    path_rotated = (rotation_matrix @ path_centered.T).T
    #scale_x = 1 / path_rotated[-1, 0]
    #scale_y = 1 / (np.max(path_rotated[:, 1]) - np.mean(path_rotated[:, 1]))
    #path_rotated[:, 0] = path_rotated[:, 0] * scale_x
    #path_rotated[:, 1] = path_rotated[:, 1] * scale_y
    return path_rotated, shift, rotation_matrix #, scale_x, scale_y

def transform_path(path, shift, rotation_matrix):
    if path.ndim == 2:
        #path[:, 0] = path[:, 0] * scale_x
        #path[:, 1] = path[:, 1] * scale_y
        return (rotation_matrix @ (path - shift).T).T
    elif path.ndim == 3:
        #path[:, :, 0] = path[:, :, 0] * scale_x
        #path[:, :, 1] = path[:, :, 1] * scale_y
        path_normalize = np.zeros(path.shape)
        for i in range(path.shape[0]):
            path_normalize[i] = (rotation_matrix @ (path[i] - shift).T).T
        return path_normalize
    else:
        raise Exception("Invalid dimension")

def closest_front_and_back(input_path, target_path):
    closest = np.zeros((target_path.shape[0], 2))
    for step in range(target_path.shape[0]):
        if step == 0:
            forward_dir = target_path[1] - target_path[0]
        else:
            forward_dir = target_path[step] - target_path[step - 1]
        
        relative_pos = input_path[:, step, :] - target_path[step]
        distances = np.linalg.norm(relative_pos, axis=1)
        forward_distances = distances[np.where(relative_pos @ forward_dir >= 0)[0]]
        backward_distances = distances[np.where(relative_pos @ forward_dir < 0)[0]]
        if len(forward_distances) == 0 or np.min(forward_distances) > NEIGHBOR_THRESHOLD:
            closest[step, 0] = DEFAULT_DISTANCE
        else:
            closest[step, 0] = np.min(forward_distances)
        
        if len(backward_distances) == 0 or np.min(backward_distances) > NEIGHBOR_THRESHOLD:
            closest[step, 1] = DEFAULT_DISTANCE
        else:
            closest[step, 1] = np.min(backward_distances)
    return closest

def generate_features(dir_name):
    pkl_list = os.listdir("./{}".format(dir_name))
    X, y, shifts, rotation_matrices, scales_x, scales_y, ID = None, None, None, None, None, None, None
    for fname in tqdm(pkl_list):
        with open("./{}/".format(dir_name) + fname, "rb") as f:
            data = pickle.load(f)
            car_mask = np.where(data["car_mask"] == 1)[0]
            pred_target = np.where(data["track_id"] == data["agent_id"])[0][0]
            if pred_target not in car_mask:
                continue
            else:
                car_mask = np.delete(car_mask, pred_target)
            all_paths = data["p_in"][car_mask]
            target_path = data["p_in"][pred_target]
            
            # feature 2 distance with respect to other cars (top 2)
            closest_distances = closest_front_and_back(all_paths, target_path)

            # feature 3 number of neighbors within 
            car_distance_per_time = np.linalg.norm((all_paths - target_path), axis=2).T
            num_neighbors = np.sum(car_distance_per_time > NEIGHBOR_THRESHOLD, axis=1).reshape(-1, 1)
            
            # feature 1 normalized location
            target_path, shift, rotation_matrix = normalize_path(target_path)
            # all_paths = transform_path(all_paths, shift, rotation_matrix, scale_x, scale_y)
            
            
            input_X = np.hstack([target_path, closest_distances, num_neighbors])
            
            if dir_name == "train":
                target_y = data["p_out"][pred_target]
                target_y = transform_path(target_y, shift, rotation_matrix)
            else:
                target_y = np.zeros([30, 2])
            
            
            input_X = np.expand_dims(input_X, axis=0)
            target_y = np.expand_dims(target_y, axis=0)
            shift = np.expand_dims(shift, axis=0)
            rotation_matrix = np.expand_dims(rotation_matrix, axis=0)
            #scale_x = np.expand_dims(scale_x, axis=0)
            #scale_y = np.expand_dims(scale_y, axis=0)
            
            if X is None:
                X = input_X
                y = target_y
                shifts = shift
                rotation_matrices = rotation_matrix
                #scales_x = scale_x
                #scales_y = scale_y
                ID = data["scene_idx"]
            else:
                X = np.vstack([X, input_X])
                y = np.vstack([y, target_y])
                shifts = np.vstack([shifts, shift])
                rotation_matrices = np.vstack([rotation_matrices, rotation_matrix])
                #scales_x = np.vstack([scales_x, scale_x])
                #scales_y = np.vstack([scales_y, scale_y])
                ID = np.append(ID, data["scene_idx"])
    if dir_name == "train":
        np.savez("processed_{}.npz".format(dir_name), X=X, y=y, shifts=shifts, rotation_matrices=rotation_matrices, ID=ID)
    else:
        np.savez("processed_{}.npz".format(dir_name), X=X, shifts=shifts, rotation_matrices=rotation_matrices, ID=ID)
    print(X.shape, y.shape, shifts.shape, rotation_matrices.shape, ID.shape)

In [4]:
generate_features("train")

100%|████████████████████████████████████████████████████████████████████████| 205942/205942 [2:22:32<00:00, 24.08it/s]


(205942, 19, 5) (205942, 30, 2) (205942, 2) (205942, 2, 2) (205942,)


In [5]:
generate_features("val_in")

100%|█████████████████████████████████████████████████████████████████████████████| 3200/3200 [00:17<00:00, 180.46it/s]

(3200, 19, 5) (3200, 30, 2) (3200, 2) (3200, 2, 2) (3200,)
